In [ ]:
"""
You can run either this notebook locally (if you have all the dependencies and a GPU) or on Google Colab.

Instructions for setting up Colab are as follows:
1. Open a new Python 3 notebook.
2. Import this notebook from GitHub (File -> Upload Notebook -> "GITHUB" tab -> copy/paste GitHub URL)
3. Connect to an instance with a GPU (Runtime -> Change runtime type -> select "GPU" for hardware accelerator)
4. Run this cell to set up dependencies.
"""
# If you're using Google Colab and not running locally, run this cell.

## Install dependencies
!pip install wget
!apt-get install sox libsndfile1 ffmpeg
!pip install text-unidecode
!pip install ipython

# ## Install NeMo
BRANCH = 'main'
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@{BRANCH}#egg=nemo_toolkit[asr]

## Install TorchAudio
!pip install torchaudio -f https://download.pytorch.org/whl/torch_stable.html

# Streaming Multitalker ASR

## Streaming Multitalker ASR with Self-Speaker Adaptation

This tutorial shows you how to use NeMo's streaming multitalker ASR system based on the approach described in [(Wang et al., 2025)](https://arxiv.org/abs/2506.22646). This system transcribes each speaker separately in multispeaker audio using speaker activity information from a streaming diarization model.

### How This Approach Works

The streaming multitalker Parakeet model uses **self-speaker adaptation**, which means:

1. **No Speaker Enrollment Required**: You only need speaker activity predictions from a diarization model (like Streaming Sortformer)
2. **Speaker Kernel Injection**: The model injects speaker-specific kernels into encoder layers to focus on each target speaker
3. **Multi-Instance Architecture**: You run one model instance per speaker, and each instance processes the same audio
4. **Handles Overlapping Speech**: Each instance focuses on one speaker, so it can transcribe overlapped speech segments

### Cache-Aware Streaming

The model uses stateful cache-based inference [(Noroozi et al., 2023)](https://arxiv.org/abs/2312.17279) for streaming:
- Left and right contexts in the encoder are constrained for low latency
- An activation caching mechanism enables the encoder to operate autoregressively during inference
- The model maintains consistent behavior between training and inference

## Multi-Instance Architecture Overview

The streaming multitalker Parakeet model employs a **multi-instance approach** where one model instance is deployed per speaker:

<img src="images/multi_instance.png" alt="Multi-instance inference of Multitalker Parakeet model" style="width: 800px;"/>

Each model instance:
- Receives the same mixed audio input
- Injects **speaker-specific kernels** generated from diarization-based speaker activity
- Produces transcription output specific to its target speaker
- Operates independently and can run in parallel with other instances

### Speaker Kernel Injection Mechanism

Learnable speaker kernels are injected into selected layers of the Fast-Conformer encoder:

<img src="images/speaker_injection.png" alt="Speaker Kernel Injection Mechanism" style="width: 800px;"/>

The speaker kernels are generated through speaker supervision activations that detect speech activity for each target speaker from the streaming diarization output. This enables the encoder states to become more responsive to the targeted speaker's speech characteristics.

## Setup and Data Preparation

In this tutorial, we will demonstrate streaming multitalker ASR using:
1. **Streaming Sortformer** for real-time speaker diarization
2. **Streaming Multitalker Parakeet** for speaker-wise ASR

Let's start by downloading a toy example audio file with multiple speakers.  

In [ ]:
import os
import wget
ROOT = os.getcwd()
data_dir = os.path.join(ROOT,'data')
os.makedirs(data_dir, exist_ok=True)
an4_audio = os.path.join(data_dir,'an4_diarize_test.wav')
an4_rttm = os.path.join(data_dir,'an4_diarize_test.rttm')
if not os.path.exists(an4_audio):
    an4_audio_url = "https://nemo-public.s3.us-east-2.amazonaws.com/an4_diarize_test.wav"
    an4_audio = wget.download(an4_audio_url, data_dir)
if not os.path.exists(an4_rttm):
    an4_rttm_url = "https://nemo-public.s3.us-east-2.amazonaws.com/an4_diarize_test.rttm"
    an4_rttm = wget.download(an4_rttm_url, data_dir)

Let's visualize the waveform and listen to the audio. You'll notice that there are two speakers in this audio clip.

In [ ]:
import IPython
import matplotlib.pyplot as plt
import numpy as np
import librosa

sr = 16000
signal, sr = librosa.load(an4_audio, sr=sr) 

fig, ax = plt.subplots(1, 1)
fig.set_figwidth(20)
fig.set_figheight(2)
plt.plot(np.arange(len(signal)), signal, 'gray')
fig.suptitle('Multispeaker Audio Waveform', fontsize=16)
plt.xlabel('time (secs)', fontsize=18)
ax.margins(x=0)
plt.ylabel('signal strength', fontsize=16)
a, _ = plt.xticks()
plt.xticks(a, a/sr)

IPython.display.Audio(signal, rate=sr)

## Step 1: Streaming Speaker Diarization

Now that we have a multispeaker audio file, the first step is to perform streaming speaker diarization using **Streaming Sortformer**. This will provide us with speaker activity information needed for self-speaker adaptation.

### Download Streaming Sortformer Diarization Model

To download the streaming Sortformer diarizer from [HuggingFace](https://huggingface.co/nvidia), you need a [HuggingFace Access Token](https://huggingface.co/docs/hub/en/security-tokens). 

Alternatively, you can download the `.nemo` file from [Streaming Sortformer HuggingFace model card](https://huggingface.co/nvidia/diar_streaming_sortformer_4spk-v2) and specify the file path.

In [ ]:
from nemo.collections.asr.models import SortformerEncLabelModel
from huggingface_hub import get_token as get_hf_token
import torch

if get_hf_token() is not None and get_hf_token().startswith("hf_"):
    # If you have logged into HuggingFace hub and have access token 
    diar_model = SortformerEncLabelModel.from_pretrained("nvidia/diar_streaming_sortformer_4spk-v2")
else:
    # You can download ".nemo" file from https://huggingface.co/nvidia/diar_streaming_sortformer_4spk-v2 and specify the path.
    diar_model = SortformerEncLabelModel.restore_from(restore_path="/path/to/diar_streaming_sortformer_4spk-v2.nemo", map_location=torch.device('cuda'), strict=False)
diar_model.eval()

### Configure Streaming Parameters for Sortformer

Set streaming parameters (all measured in 80ms frames):
- `chunk_len`: Number of frames in a processing chunk
- `chunk_right_context`: Right context length (determines latency with `chunk_len`)
- `fifo_len`: Number of previous frames from FIFO queue
- `spkcache_update_period`: Frames extracted from FIFO for speaker cache update
- `spkcache_len`: Total frames in speaker cache

The input buffer latency is determined by `chunk_len` + `chunk_right_context`.

In [ ]:
import time
import math
import torch
import torch.amp
from tqdm import tqdm 

# If cuda is available, assign the model to cuda
if torch.cuda.is_available():
    diar_model.to(torch.device("cuda"))

global autocast
autocast = torch.amp.autocast(diar_model.device.type, enabled=True)

# Set the streaming parameters corresponding to 1.04s latency setup
diar_model.sortformer_modules.chunk_len = 6
diar_model.sortformer_modules.spkcache_len = 188
diar_model.sortformer_modules.chunk_right_context = 7
diar_model.sortformer_modules.fifo_len = 188
diar_model.sortformer_modules.spkcache_update_period = 144
diar_model.sortformer_modules.log = False

# Validate that the streaming parameters are set correctly
diar_model.sortformer_modules._check_streaming_parameters()

### Feature Extraction and Streaming Diarization

Extract log-mel features from the audio signal and prepare for streaming diarization:

In [ ]:
audio_signal = torch.tensor(signal).unsqueeze(0).to(diar_model.device)
audio_signal_length = torch.tensor([audio_signal.shape[1]]).to(diar_model.device)
processed_signal, processed_signal_length = diar_model.preprocessor(input_signal=audio_signal, length=audio_signal_length)

### Run Streaming Diarization Loop

Initialize the streaming state and run the streaming diarization to get speaker activity predictions:

In [ ]:
batch_size = 1
processed_signal_offset = torch.zeros((batch_size,), dtype=torch.long, device=diar_model.device)

streaming_state = diar_model.sortformer_modules.init_streaming_state(
        batch_size=batch_size,
        async_streaming=True,
        device=diar_model.device
    )
total_preds = torch.zeros((batch_size, 0, diar_model.sortformer_modules.n_spk), device=diar_model.device)

streaming_loader = diar_model.sortformer_modules.streaming_feat_loader(
    feat_seq=processed_signal,
    feat_seq_length=processed_signal_length,
    feat_seq_offset=processed_signal_offset,
)

num_chunks = math.ceil(
    processed_signal.shape[2] / (diar_model.sortformer_modules.chunk_len * diar_model.sortformer_modules.subsampling_factor)
)

print(f"Processing {num_chunks} chunks for diarization...")

In [ ]:
# Run streaming diarization
for i, chunk_feat_seq_t, feat_lengths, left_offset, right_offset in tqdm(
    streaming_loader,
    total=num_chunks,
    desc="Streaming Diarization",
    disable=False,
):
    with torch.inference_mode():
        with autocast:
            streaming_state, total_preds = diar_model.forward_streaming_step(
                processed_signal=chunk_feat_seq_t,
                processed_signal_length=feat_lengths,
                streaming_state=streaming_state,
                total_preds=total_preds,
                left_offset=left_offset,
                right_offset=right_offset,
            )

print(f"Diarization complete! Total predictions shape: {total_preds.shape}")

### Visualize Diarization Results

Let's visualize the speaker diarization predictions:

In [ ]:
def plot_diarout(preds):
    """Visualize diarization predictions"""
    preds_mat = preds.cpu().numpy().transpose()
    cmap_str, grid_color_p = 'viridis', 'gray'
    LW, FS = 0.4, 36

    yticklabels = ["spk0", "spk1", "spk2", "spk3"]
    yticks = np.arange(len(yticklabels))
    fig, axs = plt.subplots(1, 1, figsize=(30, 3)) 

    axs.imshow(preds_mat, cmap=cmap_str, interpolation='nearest') 
    axs.set_title('Diarization Predictions (Speaker Activity)', fontsize=FS)
    axs.set_xticks(np.arange(-.5, preds_mat.shape[1], 1), minor=True)
    axs.set_yticks(yticks)
    axs.set_yticklabels(yticklabels)
    axs.set_xlabel(f"80 ms Frames", fontsize=FS)
    axs.grid(which='minor', color=grid_color_p, linestyle='-', linewidth=LW)
    plt.show()

plot_diarout(total_preds[0,:])

## Step 2: Streaming Multitalker ASR with Self-Speaker Adaptation

Now that we have speaker activity information from diarization, we can load the streaming multitalker Parakeet model and perform speaker-wise ASR.

### Download Streaming Multitalker Parakeet Model

In [ ]:
from nemo.collections.asr.models import ASRModel
import torch
    
if get_hf_token() is not None and get_hf_token().startswith("hf_"):
    # If you have logged into HuggingFace hub and have access token 
    asr_model = ASRModel.from_pretrained("nvidia/multitalker-parakeet-streaming-0.6b-v1")
else:
    # You can download ".nemo" file from https://huggingface.co/nvidia/multitalker-parakeet-streaming-0.6b-v1 and specify the path.
    asr_model = ASRModel.restore_from(restore_path="/path/to/multitalker-parakeet-streaming-0.6b-v1.nemo", map_location=torch.device('cuda'))
diar_model.eval()

asr_model.eval()
if torch.cuda.is_available():
    asr_model.to(torch.device("cuda"))
    
print("ASR Model loaded successfully!")

### Configure Cache-Aware Streaming Parameters

Set the streaming attention context size for the ASR model. The latency is determined by the attention context configuration (measured in 80ms frames):

- `[70, 0]`: Chunk size = 1 (1 * 80ms = 0.08s)  
- `[70, 1]`: Chunk size = 2 (2 * 80ms = 0.16s)  
- `[70, 6]`: Chunk size = 7 (7 * 80ms = 0.56s)  
- `[70, 13]`: Chunk size = 14 (14 * 80ms = 1.12s)

In [ ]:
# Set streaming parameters for 1.12s latency (matching diarization)
att_context_size = [70, 13]  # [left_context, right_context] in frames


print(f"ASR streaming configured with attention context: {att_context_size}")


### Determine Number of Active Speakers

Analyze the diarization output to determine how many speakers are active in the audio:

## Step 3: Multi-Instance Streaming ASR

Now we'll run streaming multitalker ASR using the multi-instance architecture. We'll create one model instance per detected speaker.

### Step 3-1: Prepare the configurations



Since streaming processing of speech signals involves many complications in cache handling, we first need to set up a config dataclass that aggregates all the parameters in one place. You can access this class in the example multitalker streaming ASR script: [speech_to_text_multitalker_streaming_infer.py](https://raw.githubusercontent.com/NVIDIA-NeMo/NeMo/main/examples/asr/asr_cache_aware_streaming/speech_to_text_multitalker_streaming_infer.py") 

In [ ]:
from dataclasses import dataclass, field
from typing import List, Optional

@dataclass
class MultitalkerTranscriptionConfig:
    """
    Configuration for Multi-talker transcription with an ASR model and a diarization model.
    """
    # Required configs
    diar_model: Optional[str] = None
    diar_pretrained_name: Optional[str] = None
    max_num_of_spks: Optional[int] = 4
    parallel_speaker_strategy: bool = True
    masked_asr: bool = True
    mask_preencode: bool = False
    cache_gating: bool = True
    cache_gating_buffer_size: int = 2
    single_speaker_mode: bool = False
    feat_len_sec: float = 0.01

    # General configs
    session_len_sec: float = -1
    num_workers: int = 8
    random_seed: Optional[int] = None
    log: bool = True

    # Streaming diarization configs
    streaming_mode: bool = True
    spkcache_len: int = 188
    spkcache_refresh_rate: int = 0
    fifo_len: int = 188
    chunk_len: int = 0
    chunk_left_context: int = 0
    chunk_right_context: int = 0

    # If `cuda` is a negative number, inference will be on CPU only.
    cuda: Optional[int] = None
    allow_mps: bool = False  
    matmul_precision: str = "highest"  # Literal["highest", "high", "medium"]

    # ASR Configs
    asr_model: Optional[str] = None
    device: str = 'cuda'
    audio_file: Optional[str] = None
    manifest_file: Optional[str] = None
    att_context_size: Optional[List[int]] = field(default_factory=lambda: [70, 13])
    use_amp: bool = True
    debug_mode: bool = False
    deploy_mode: bool = False
    batch_size: int = 32
    chunk_size: int = -1
    shift_size: int = -1
    left_chunks: int = 2
    online_normalization: bool = False
    output_path: Optional[str] = None
    pad_and_drop_preencoded: bool = False
    set_decoder: Optional[str] = None  # ["ctc", "rnnt"]
    generate_realtime_scripts: bool = False
    spk_supervision: str = "diar"  # ["diar", "rttm"]
    binary_diar_preds: bool = False

    # Multitalker transcription configs
    verbose: bool = False
    word_window: int = 50
    sent_break_sec: float = 30.0
    fix_prev_words_count: int = 5
    update_prev_words_sentence: int = 5
    left_frame_shift: int = -1
    right_frame_shift: int = 0
    min_sigmoid_val: float = 1e-2
    discarded_frames: int = 8
    print_time: bool = True
    print_sample_indices: List[int] = field(default_factory=lambda: [0])
    colored_text: bool = True
    real_time_mode: bool = False
    print_path: Optional[str] = None
    ignored_initial_frame_steps: int = 5
    finetune_realtime_ratio: float = 0.01

Using this dataclass for configurations, assign the designated streaming speaker diarization parameters to a diarization model. This process ensures that the processing window size and cache sizes are synchronized.

In [ ]:
from omegaconf import OmegaConf
# Create configuration object for multitalker transcription
cfg = MultitalkerTranscriptionConfig()
# Convert dataclass to OmegaConf DictConfig so it has .get() method
cfg = OmegaConf.structured(cfg)

cfg.att_context_size = [70,13]
cfg.output_path = "/path/to/output.json"
cfg.audio_file = an4_audio
print(f"ASR streaming configured with attention context: {att_context_size}")

for key in cfg:
    cfg[key] = None if cfg[key] == 'None' else cfg[key]

# Set streaming mode diar_model params (matching the diarization setup from lines 263-271 of reference file)
diar_model.streaming_mode = cfg.streaming_mode
diar_model.sortformer_modules.chunk_len = cfg.chunk_len if cfg.chunk_len > 0 else 6
diar_model.sortformer_modules.spkcache_len = cfg.spkcache_len
diar_model.sortformer_modules.chunk_left_context = cfg.chunk_left_context
diar_model.sortformer_modules.chunk_right_context = cfg.chunk_right_context if cfg.chunk_right_context > 0 else 7
diar_model.sortformer_modules.fifo_len = cfg.fifo_len
diar_model.sortformer_modules.log = cfg.log
diar_model.sortformer_modules.spkcache_refresh_rate = cfg.spkcache_refresh_rate

# Set online normalization flag
online_normalization = cfg.online_normalization

# Set pad_and_drop_preencoded flag
pad_and_drop_preencoded = cfg.pad_and_drop_preencoded

print(f"Configuration setup complete!")
print(f"Audio file: {cfg.audio_file}")
print(f"Streaming mode: {diar_model.streaming_mode}")
print(f"Diar model chunk_len: {diar_model.sortformer_modules.chunk_len}")
print(f"Diar model chunk_right_context: {diar_model.sortformer_modules.chunk_right_context}")
print(f"Online normalization: {online_normalization}")

### Run Multi-Instance Streaming ASR

For each active speaker, we'll run a separate ASR model instance with speaker-specific kernel injection. In practice, these instances can run in parallel.

For testing purposes, first set up a single-sample batch and feed it into the streaming buffer. The streaming buffer simulates an input audio stream so that we can run the multitalker ASR model in a streaming manner.

In [ ]:
from nemo.collections.asr.parts.utils.streaming_utils import CacheAwareStreamingAudioBuffer

samples = [
    {
        'audio_filepath': cfg.audio_file,
    }
]
streaming_buffer = CacheAwareStreamingAudioBuffer(
    model=asr_model,
    online_normalization=online_normalization,
    pad_and_drop_preencoded=cfg.pad_and_drop_preencoded,
)
streaming_buffer.append_audio_file(audio_filepath=cfg.audio_file, stream_id=-1)

### Step 3-2: Run Multitalker ASR with the prepared configurations and streaming buffer



In [ ]:
from nemo.collections.asr.parts.utils.multispk_transcribe_utils import SpeakerTaggedASR
from pprint import pprint
speaker_transcriptions = {}

streaming_buffer_iter = iter(streaming_buffer)
multispk_asr_streamer = SpeakerTaggedASR(cfg, asr_model, diar_model)
feat_frame_count = 0
print(f"length of streaming_buffer_iter: {len(streaming_buffer)} {time.time()}")

for step_num, (chunk_audio, chunk_lengths) in enumerate(streaming_buffer_iter):
    drop_extra_pre_encoded = (
        0
        if step_num == 0 and not pad_and_drop_preencoded
        else asr_model.encoder.streaming_cfg.drop_extra_pre_encoded
    )
    loop_start_time = time.time()
    with torch.inference_mode():
        with autocast:
            with torch.no_grad():
                multispk_asr_streamer.perform_parallel_streaming_stt_spk(
                    step_num=step_num,
                    chunk_audio=chunk_audio,
                    chunk_lengths=chunk_lengths,
                    is_buffer_empty=streaming_buffer.is_buffer_empty(),
                    drop_extra_pre_encoded=drop_extra_pre_encoded,
                )
                pprint(multispk_asr_streamer.instance_manager.batch_asr_states[0].seglsts)
                
seglst_dict_list = multispk_asr_streamer.generate_seglst_dicts_from_parallel_streaming(samples=samples)

from pprint import pprint
print(f"SegLST style multispeaker transcription\n {seglst_dict_list}")

## Results and Analysis

### Display Speaker-Wise Transcriptions

Let's display the final transcriptions for each speaker: 

In [ ]:
print("\n" + "="*80)
print("Final transcriptions with speaker tagging and timestamps")
print("="*80 + "\n")

# Display SegLST dict list with timestamps and speaker-tagged transcriptions
# Format: {'speaker': 'speaker_0', 'start_time': 2.64, 'end_time': 5.6, 'words': 'eleven twenty seven fifty seven', 'session_id': 'an4_diarize_test'}
if seglst_dict_list:
    for idx, seglst in enumerate(seglst_dict_list):
        speaker = seglst.get('speaker', 'Unknown')
        start_time = seglst.get('start_time', 0.0)
        end_time = seglst.get('end_time', 0.0)
        words = seglst.get('words', '')
        session_id = seglst.get('session_id', '')
        
        print(f"[{idx+1}] {speaker} ({start_time:.2f}s - {end_time:.2f}s): {words}")
    
    print(f"\n{'-'*80}")
    print(f"Total segments: {len(seglst_dict_list)}")
else:
    print("No transcriptions available in seglst_dict_list.")


This tutorial demonstrated streaming multitalker ASR using self-speaker adaptation with NeMo:

### Key Components

1. **Streaming Sortformer Diarization**: Provides real-time speaker activity predictions using arrival-order speaker cache (AOSC)
2. **Cache-Aware Streaming ASR**: FastConformer-based model with stateful cache-based inference for low-latency transcription
3. **Self-Speaker Adaptation**: Speaker kernels injected into encoder layers based on diarization, enabling speaker-focused recognition without enrollment
4. **Multi-Instance Architecture**: One model instance per speaker, enabling parallel processing and handling of severe speech overlap

### Advantages

- **No enrollment required**: Only needs diarization output, no pre-recorded speaker audio
- **Handles overlap**: Each instance focuses on one speaker, even during fully overlapped speech
- **Streaming capable**: Real-time processing with configurable latency (0.08s to 1.12s+)
- **State-of-the-art performance**: Achieves strong results on challenging multitalker benchmarks

### Configuration Summary

In this tutorial, we used:
- **Diarization latency**: 1.04s (chunk_len=6, chunk_right_context=7)
- **ASR latency**: 1.12s (att_context_size=[70, 13])
- **Number of speakers**: Automatically detected from diarization output

## References

[1] [Speaker Targeting via Self-Speaker Adaptation for Multi-talker ASR](https://arxiv.org/abs/2506.22646)  


[2] [Stateful Conformer with Cache-based Inference for Streaming Automatic Speech Recognition](https://arxiv.org/abs/2312.17279)  

[3] [Streaming Sortformer: Speaker Cache-Based Online Speaker Diarization with Arrival-Time Ordering](https://arxiv.org/abs/2507.18446)

[4] [NEST: Self-supervised Fast Conformer as All-purpose Seasoning to Speech Processing Tasks](https://arxiv.org/abs/2408.13106)

[5] [Fast Conformer with Linearly Scalable Attention for Efficient Speech Recognition](https://arxiv.org/abs/2305.05084)